In [1]:
import requests
from bs4 import BeautifulSoup
import sqlite3
%run Definitions.py
from datetime import datetime, date, timedelta
import re
import os
from collections import *
import pandas as pd
import shutil

In [2]:
# Function to extract tables from XML content and convert to data frames
def extract_tables(xml_content):
    soup = BeautifulSoup(xml_content, 'xml')
    tables = soup.find_all('infoTable')
    data = []
    for table in tables:
        row = {}
        for element in table:
            cleaned_text = element.text.strip()
            try:
                # Decode the text using UTF-8 encoding and ignore any errors
                cleaned_text = cleaned_text.encode('utf-8').decode('utf-8', 'ignore')
            except UnicodeDecodeError:
                pass
            
            if element.name == 'shrsOrPrnAmt':
                # Remove '\n' from shrsOrPrnAmt column
                cleaned_text = cleaned_text.replace('\n', '')
            
            if element.name == 'votingAuthority':
                # Remove '\n0\n0' from the last column
                cleaned_text = cleaned_text.replace('\n0\n0', '')
            
            row[element.name] = cleaned_text
        data.append(row)

    df = pd.DataFrame(data)
    return df

# Function to split shrsOrPrnAmt column into two separate columns
def split_shrsOrPrnAmt(df):
    df[['shrsOrPrnAmt.sshPrnamt', 'shrsOrPrnAmt.sshPrnamtType']] = df['shrsOrPrnAmt'].str.extract(r'(\d+)(.*)')
    df['shrsOrPrnAmt.sshPrnamt'] = df['shrsOrPrnAmt.sshPrnamt'].str.replace(',', '')  # Remove commas from numbers
    df.drop('shrsOrPrnAmt', axis=1, inplace=True)
    return df

# Function to extract desired variables from filing text
def extract_filing_info(xml_content):
    soup = BeautifulSoup(xml_content, 'xml')

    # Extract the accession number
    sec_document_tag = soup.find('SEC-DOCUMENT')
    if sec_document_tag is None:
        print("Error: <sec-document> tag not found.")
        return None, None, None, None

    first_line = sec_document_tag.text.strip().split('\n')[0]
    accession_number = first_line.split(':')[0].strip()

    # Remove ".txt" extension from the accession number
    accession_number = accession_number.replace('.txt', '')

    # print(accession_number)

    # Extract CIK (Central Index Key)
    cik = soup.find('cik').text.strip()

    # Extract filing manager's name
    filing_manager_name = soup.find('filingManager').find('name').text.strip()

    # Extract period of report
    period_of_report = soup.find('periodOfReport').text.strip()

    # Parse the date string to a datetime object
    period_of_report = datetime.strptime(period_of_report, '%m-%d-%Y')

    # Format the date object as SQLite date string
    period_of_report = period_of_report.strftime('%Y-%m-%d')

    return accession_number, cik, filing_manager_name, period_of_report


In [3]:
# ### USE THIS CODE TO CONVERT date_of_report column in filings from the '%m-%d-%Y' format to the '%Y-%m-%d' format so they can be recognized as dates.

# # Connect to the SQLite database
# conn = sqlite3.connect("/Users/ralph/Biotech/BiotechDatabase.db") 
# cursor = conn.cursor()

# # Fetch all rows from the filings table
# cursor.execute("SELECT filing_id, period_of_report FROM filings")
# rows = cursor.fetchall()

# # Iterate over the rows and update the period_of_report values
# for row in rows:
#     filing_id, period_of_report = row

#     # Check if the period_of_report is already in the desired format
#     if len(period_of_report) == 10:
#         try:
#             datetime.strptime(period_of_report, '%Y-%m-%d')
#             # Skip this row as the date is already in the desired format
#             continue
#         except ValueError:
#             pass

#     # Parse the date string to a datetime object
#     date_object = datetime.strptime(period_of_report, '%m-%d-%Y')

#     # Format the date object as SQLite date string
#     sqlite_date = date_object.strftime('%Y-%m-%d')

#     # Update the period_of_report value in the table
#     cursor.execute("UPDATE filings SET period_of_report = ? WHERE filing_id = ?", (sqlite_date, filing_id))

# # Commit the changes and close the connection
# conn.commit()
# conn.close()

In [4]:
# ### USE THIS SECTION IF YOU NEED TO DELETE SOME ACCESSION ROWS THAT WERE PARSED BY MISTAKE (LIKE 13F-HR/A FILES)

# directory = "data/13F"
# # processed_directory = 'data/13G_processed'
# # # Create the processed directory if it doesn't exist
# # os.makedirs(processed_directory, exist_ok=True)

# # Get a list of subfolders
# subfolders = sorted([f for f in os.listdir(directory) if os.path.isdir(os.path.join(directory, f))], reverse = True)
# # subfolders = subfolders[3:4]

# print(subfolders)

# # Iterate over the subfolders
# for subfolder in subfolders:
#     print(f'Year being cleaned: {subfolder}')
#     subdirectory = os.path.join(directory, subfolder)
#     # processed_subdirectory = os.path.join(processed_directory, subfolder)
#     # # Create the processed subdirectory if it doesn't exist
#     # os.makedirs(processed_subdirectory, exist_ok=True)
    
#     # Create a list of filingIDs from the data directory
#     filingIDs = []
#     for filename in os.listdir(subdirectory):
#         if filename.endswith(".txt"):
#             filingID = os.path.splitext(filename)[0]
#             filingIDs.append(filingID)
#     print(f'Filings in year {subfolder}: {len(filingIDs)}')

#     # Connect to the SQLite database and find already parsed filings
#     conn = sqlite3.connect("/Users/ralph/Biotech/BiotechDatabase.db") 
#     cursor = conn.cursor()
#     # Get a list of all 13F filingIDs from the SQL table

#     cursor.execute("SELECT accession FROM SEC_13F_filings")

#     sql_filingIDs_all = [row[0] for row in cursor.fetchall()]

#     filings_to_delete = list(set(filingIDs) - set(sql_filingIDs_all))

#     num_rows_deleted = 0
#     num_rows_deleted2 = 0
#     # Delete rows where filing_id is in filings_to_delete

#     for filing_id in filings_to_delete:
#         cursor.execute("DELETE FROM filings WHERE filing_id = ?", (filing_id,))
#         num_rows_deleted += cursor.rowcount
#         cursor.execute("DELETE FROM holdings WHERE filing_id = ?", (filing_id,))
#         num_rows_deleted2 += cursor.rowcount
#     cursor.close()
#     conn.commit()
#     conn.close()
#     print(f'Number of rows deleted from filings: {num_rows_deleted}')
#     print(f'Number of rows deleted from holdings: {num_rows_deleted2}')


In [5]:


directory = "data/13F"
# processed_directory = 'data/13G_processed'
# # Create the processed directory if it doesn't exist
# os.makedirs(processed_directory, exist_ok=True)

# Get a list of subfolders
subfolders = sorted([f for f in os.listdir(directory) if os.path.isdir(os.path.join(directory, f))], reverse = True)
subfolders = subfolders

# print(subfolders)

# Iterate over the subfolders
for subfolder in subfolders[0:2]:
    print(f'Year being parsed: {subfolder}')
    subdirectory = os.path.join(directory, subfolder)
    # processed_subdirectory = os.path.join(processed_directory, subfolder)
    # # Create the processed subdirectory if it doesn't exist
    # os.makedirs(processed_subdirectory, exist_ok=True)
    
    # Create a list of filingIDs from the data directory
    filingIDs = []
    for filename in os.listdir(subdirectory):
        if filename.endswith(".txt"):
            filingID = os.path.splitext(filename)[0]
            filingIDs.append(filingID)
    print(f'Filings in year {subfolder}: {len(filingIDs)}')


    # Connect to the SQLite database and find already parsed filings
    conn = sqlite3.connect("/Users/ralph/Biotech/BiotechDatabase.db") 
    cursor = conn.cursor()
    # Get a list of parsed 13F filingIDs from the SQL table
    cursor.execute("SELECT accession FROM SEC_13F_filings WHERE [Parsed 13F] IS NULL OR [Parsed 13F] = ''")
    sql_filingIDs = [row[0] for row in cursor.fetchall()]

    cursor.execute("SELECT accession FROM SEC_13F_filings")
    sql_filingIDs_all = [row[0] for row in cursor.fetchall()]

    filings_to_parse = list(set(filingIDs).intersection(set(sql_filingIDs)))
    print(f'Filings to parse from intersection: {len(filings_to_parse)}')
    filings_to_parse.extend(list(set(filingIDs) - set(sql_filingIDs_all)))

    print(f'Filings to be parsed in year {subfolder}: {len(filings_to_parse)}')


    filing_id_file_with_exception = []
    today = date.today()

    # Iterate over the files in the directory
    for filingID in filings_to_parse:
        filename = filingID + '.txt'
        file_path = os.path.join(subdirectory, filename)

        # Read the file content as UTF-8 encoded text
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()

        # Extract tables from the file content
        df = extract_tables(content)
        if df.empty:
            print(f"No data was generated for file: {filename}.")
            continue

        # Split shrsOrPrnAmt column into two separate columns
        df = split_shrsOrPrnAmt(df)

        if 'nameOfIssuer' in df.columns:
            df.rename(columns={'nameOfIssuer': 'name_of_issuer'}, inplace=True)

        if 'titleOfClass' in df.columns:
            df.rename(columns={'titleOfClass': 'title_of_class'}, inplace=True)

        if 'putCall' in df.columns:
            df.rename(columns={'putCall': 'put_call'}, inplace=True)
        
        if 'shrsOrPrnAmt.sshPrnamt' in df.columns:
            df.rename(columns={'shrsOrPrnAmt.sshPrnamt': 'shares'}, inplace=True)


        columns_to_remove = ['investmentDiscretion', 'votingAuthority', 'shrsOrPrnAmt.sshPrnamtType', 'figi', 'otherManager']
        existing_columns = [column for column in columns_to_remove if column in df.columns]
        df = df.drop(columns=existing_columns)

        df = df.loc[:, df.columns.notnull()]
        df['filing_id'] = filingID
        df['cusip'] = df['cusip'].str.upper()
        df['name_of_issuer'] = df['name_of_issuer'].str.upper()
        

        # Extract desired filing information
        filing_id, cik, filing_manager_name, period_of_report = extract_filing_info(content)

        # # Print the extracted information
        # print("Accession Number:", filing_id)
        # print("CIK:", cik)
        # print("Filing Manager Name:", filing_manager_name)
        # print("Period of Report:", period_of_report)

        filings_sql = """
                    INSERT OR REPLACE INTO filings (
                        filing_id, 
                        cik,
                        filer_name,
                        period_of_report
                    ) 
                    VALUES (?,?,?,?)
                """

        try:
            conn = sqlite3.connect("/Users/ralph/Biotech/BiotechDatabase.db") 
            cur = conn.cursor()
            #### Insert filings table data
            filing_values = (
                filing_id,
                cik, 
                filing_manager_name.upper(), # convert all names to upper case to make unique search easier
                period_of_report
            )
            
            cur.execute(filings_sql, filing_values)
            cur.execute("UPDATE SEC_filings SET 'Parsed 13F' = ? WHERE accession = ?", (today, filingID))
            conn.commit()

            ##### Insert holdings data
            # creating column list for insertion
            cols = "`,`".join([str(i) for i in df.columns.tolist()])

            # Insert DataFrame recrds one by one.
            for index,row in df.iterrows():
                try:
                    cur = conn.cursor()
                    sql = " INSERT OR REPLACE INTO `holdings` (`" +cols + "`) VALUES (" + "?,"*(len(row)-1) + "?)"
                    cur.execute(sql, tuple(row))
                    # the connection is not autocommitted by default, so we must commit to save our changes
                    # connection.commit()
                    conn.commit()
                    # cur.close()
                except Exception as error:
                    print("Failed to add row", error)
                    filing_id_file_with_exception.append(row.loc['filing_id'])
                    print(index,row)
                    pass

                
        except Exception as error:
            print("Failed to add file", error)
        finally:
            if (conn):
                conn.close()


Year being parsed: 2023
Filings in year 2023: 20847
Filings to parse from intersection: 0
Filings to be parsed in year 2023: 1772
Year being parsed: 2022
Filings in year 2022: 27496
Filings to parse from intersection: 0
Filings to be parsed in year 2022: 0
